In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import requests

# Data Extraction and Cleaning

First, we import the necessary files. This includes the network service provider information (saved as `nsp_info`), and generation by fuel type (saved as `fuel_gen`).

## NSP Data

In [80]:
nsp_info = pd.read_csv('https://www.emi.ea.govt.nz/Wholesale/Datasets/MappingsAndGeospatial/NetworkSupplyPointsTable/20221217_NetworkSupplyPointsTable.csv')

In [81]:
nsp_info

,Current flag,NSP,NSP replaced by,POC code,Network participant,Embedded under POC code,Embedded under network participant,Reconciliation type,X flow,I flow,...,Start date,Start TP,End date,End TP,SB ICP,Balancing code,MEP,Responsible participant,Certification expiry,Metering information exemption expiry date
0,1,ABY0111ALPEGN,NaN,ABY0111,ALPE,NaN,NaN,GN,1,1,...,2012-01-01,1,NaN,NaN,NaN,CENTRALALPEG,TPNZ,TPNZ,2023-02-12,NaN
1,1,AKK0011SMRTEN,NaN,AKK0011,SMRT,KOE1101,TOPE,EN,1,0,...,2021-08-01,1,NaN,NaN,NaN,AKK0011SMRTE,AMCI,SMRT,2026-07-29,NaN
2,1,AKL0331AIALEN,NaN,AKL0331,AIAL,MNG0331,VECT,EN,1,1,...,2022-10-01,1,NaN,NaN,1001136290AA143,AKL0331AIALE,AMCI,AIAL,2022-12-24,NaN
3,1,ALB0331UNETGN,NaN,ALB0331,UNET,NaN,NaN,GN,1,1,...,2020-08-14,1,NaN,NaN,NaN,NORTHRNUNETG,TPNZ,TPNZ,2025-06-29,NaN
4,1,ALB1101UNETGN,NaN,ALB1101,UNET,NaN,NaN,GN,1,0,...,2008-05-01,1,NaN,NaN,NaN,NORTHRNUNETG,TPNZ,TPNZ,2025-04-23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,0,WWC0011WFNZEN,WWC0011TENCEN,WWC0011,WFNZ,HEP0331,UNET,EN,1,0,...,2011-01-01,1,2017-06-30,48.0,0002220000WFEDE,WWC0011WFNZE,AMCI,WFNZ,2017-12-14,NaN
2210,0,WWD1102MERIGD,WWD1102MERIGG,WWD1102,MERI,NaN,NaN,GD,1,0,...,2008-12-01,1,2009-02-16,48.0,NaN,WWD1102MERIG,MERG,MERI,2023-08-11,NaN
2211,0,WWD1103MERIGD,WWD1103MERIGG,WWD1103,MERI,NaN,NaN,GD,1,0,...,2008-12-01,1,2009-02-16,48.0,NaN,WWD1103MERIG,MERG,MERI,2023-08-12,NaN
2212,0,WWK0111WAIKEN,WWK0111WAIKEN,WWK0111,WAIK,WRK0331,HAWK,EN,1,0,...,2005-09-01,1,2008-04-30,48.0,NaN,WWK0111WAIKE,NaN,NaN,NaN,NaN


In [82]:
nsp_info.columns.tolist()

['Current flag',
 'NSP',
 'NSP replaced by',
 'POC code',
 'Network participant',
 'Embedded under POC code',
 'Embedded under network participant',
 'Reconciliation type',
 'X flow',
 'I flow',
 'Description',
 'NZTM easting',
 'NZTM northing',
 'Network reporting region ID',
 'Network reporting region',
 'Zone',
 'Island',
 'Start date',
 'Start TP',
 'End date',
 'End TP',
 'SB ICP',
 'Balancing code',
 'MEP',
 'Responsible participant',
 'Certification expiry',
 'Metering information exemption expiry date']

We can see this dataset contains a lot of information about the network service providers (NSPs). The only columns relevant for us are `POC code`, `NZTM easting`, and `NZTM northing`. POC stands for point of connection on the electrcitiy grid. Some NSPs have the same POC code because they consist of multiple generation units within the same facility. NZTM easting and northing describe the location of the POC in the NZTM coordinate system. We can remove the unnecessary columns.

In [83]:
nsp_info = nsp_info[['POC code', 'NZTM easting', 'NZTM northing']].copy()

In [84]:
nsp_info[nsp_info['NZTM easting'].isna()]

,POC code,NZTM easting,NZTM northing
1,AKK0011,NaN,NaN
25,BCK0011,NaN,NaN
26,BCK0012,NaN,NaN
30,BJL0011,NaN,NaN
34,BMR0011,NaN,NaN
...,...,...,...
2057,VWC0011,NaN,NaN
2058,VWC0011,NaN,NaN
2059,VWC0011,NaN,NaN
2117,WPH0011,NaN,NaN


Notice there are several POCs without NZTM coordinates. These POCs are usually either no longer operating or are embedded under another POC. As a result, we can disregard these POCs without losing any information.

In [85]:
nsp_info.dropna(inplace=True)

Another point to note is that several generation units can have the same POC code if they are operating within the same facility. We can see this by comparing the length of the dataset with the length of unique values.

In [86]:
print("Total data points: {count}".format(count=len(nsp_info)))

print("Unique data points: {count}".format(count=len(nsp_info.value_counts())))

Total data points: 1906
Unique data points: 485


There were an astounding number of duplicates in our data. We can drop these without any loss of information.

In [87]:
nsp_info.drop_duplicates(inplace=True)

In [88]:
nsp_info

,POC code,NZTM easting,NZTM northing
0,ABY0111,1424393.0,5097839.0
2,AKL0331,1762701.0,5907699.0
3,ALB0331,1750900.0,5932739.0
4,ALB1101,1750900.0,5932739.0
5,ANI0331,1931640.0,5754616.0
...,...,...,...
2155,WRA0111,1981648.0,5674331.0
2158,WRA0501,1981648.0,5674331.0
2197,WTN0111,1237336.0,4879291.0
2198,WTN0661,1237336.0,4879291.0


This looks good for the moment, let's export this data so we have it saved if needed elsewhere.

In [89]:
nsp_info.to_csv('data/nsp_info.csv', index=False)

## Fuel Generation Data

In [73]:
fuel_gen = pd.read_csv('https://www.emi.ea.govt.nz/Wholesale/Datasets/Generation/Generation_MD/202211_Generation_MD.csv')

In [78]:
fuel_gen

,Site_Code,POC_Code,Nwk_Code,Gen_Code,Fuel_Code,Tech_Code,Trading_Date,TP1,TP2,TP3,...,TP41,TP42,TP43,TP44,TP45,TP46,TP47,TP48,TP49,TP50
0,ARA,ARA2201,MRPL,aratiatia,Hydro,Hydro,2022-11-01,23640,23590,30130,...,39070,39060,38950,39100,39010,38960,39040,38960,NaN,NaN
1,ARA,ARA2201,MRPL,aratiatia,Hydro,Hydro,2022-11-02,37340,24300,21050,...,37510,36780,36970,36920,36980,36870,36900,26000,NaN,NaN
2,ARA,ARA2201,MRPL,aratiatia,Hydro,Hydro,2022-11-03,25040,15960,10370,...,36520,36410,36480,34060,24770,18960,15440,11810,NaN,NaN
3,ARA,ARA2201,MRPL,aratiatia,Hydro,Hydro,2022-11-04,11830,12730,8000,...,23840,24620,24830,25140,25120,25250,24020,12300,NaN,NaN
4,ARA,ARA2201,MRPL,aratiatia,Hydro,Hydro,2022-11-05,12330,12300,12320,...,22680,22600,22670,22680,23600,23640,23630,23600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,WWD,WWD1103,MERI,west_wind,Wind,Wind,2022-11-26,10451,12149,8548,...,29315,27086,25050,27218,28169,28079,28025,24999,NaN,NaN
2256,WWD,WWD1103,MERI,west_wind,Wind,Wind,2022-11-27,21975,23230,28269,...,2720,1146,297,3718,4911,10490,19926,27001,NaN,NaN
2257,WWD,WWD1103,MERI,west_wind,Wind,Wind,2022-11-28,27339,26956,28241,...,15251,19838,18063,16951,16498,18356,24159,28895,NaN,NaN
2258,WWD,WWD1103,MERI,west_wind,Wind,Wind,2022-11-29,32248,33141,33195,...,32352,30964,30027,30044,30767,28349,28680,29829,NaN,NaN


In [76]:
fuel_gen.columns

Index(['Site_Code', 'POC_Code', 'Nwk_Code', 'Gen_Code', 'Fuel_Code',
       'Tech_Code', 'Trading_Date', 'TP1', 'TP2', 'TP3', 'TP4', 'TP5', 'TP6',
       'TP7', 'TP8', 'TP9', 'TP10', 'TP11', 'TP12', 'TP13', 'TP14', 'TP15',
       'TP16', 'TP17', 'TP18', 'TP19', 'TP20', 'TP21', 'TP22', 'TP23', 'TP24',
       'TP25', 'TP26', 'TP27', 'TP28', 'TP29', 'TP30', 'TP31', 'TP32', 'TP33',
       'TP34', 'TP35', 'TP36', 'TP37', 'TP38', 'TP39', 'TP40', 'TP41', 'TP42',
       'TP43', 'TP44', 'TP45', 'TP46', 'TP47', 'TP48', 'TP49', 'TP50'],
      dtype='object')